# TP 2 : récursivité et conditions

Dans tout le TP, il est interdit d'utiliser des boucles (pas de `for` ni `while`).

Rappel : pensez à tester toutes vos fonctions sur des exemples.

## Petites questions

1. Définir une fonction `divise` telle que `divise a b` renvoie `true` si  `a` divise `b`, `false` sinon. Il est interdit d'utiliser `if`.
2. Définir une fonction récursive `somme` telle que `somme n` renvoie $\sum_{k=1}^n k^2$.
3. Définir une fonction récursive `u` telle que `u n` renvoie $u_n$ définie par :
$$u_0 = 42$$
$$u_{n} = 3\sqrt{u_{n - 1}} + 2$$
4. Définir une fonction récursive `v` telle que `v a n` renvoie $v_n$ définie par (on fera attention à n'utiliser qu'un appel récursif) :
$$v_0 = a$$
$$v_{n + 1} = \frac{1}{2}(v_n + \frac{a}{v_n})$$
5. Calculer `v a n` pour plusieurs valeurs de `a` et de `n` : que peut-on conjecturer sur la limite de $v_n$ ? Optionnel : le démontrer.  
6. Soient $a$ et $b$ deux entiers et $q, r$ le quotient et reste de la division euclidienne de $a$ par $b$ ($a = bq + r$). En utilisant le fait que $PGCD(a, b)$ = $PGCD(b, r)$, écrire une fonction `euclide` telle que `euclide a b` renvoie le PGCD de $a$ et $b$.

In [6]:
(*question 1*)
let divise a b =
    a mod b = 0;;

divise 12 6;;
divise 13 12;;

val divise : int -> int -> bool = <fun>


- : bool = true


- : bool = false


In [5]:
(*question 2*)
let rec somme n =
    if n = 1 then 1
    else n*n + somme(n-1);;
    
somme 5;;

val somme : int -> int = <fun>


- : int = 55


In [4]:
(*question 3*)
let rec u n =
    if n = 0 then 42.
    else 3.*.(u (n-1))**0.5 +. 2.;;

u 4;;

val u : int -> float = <fun>


- : float = 13.2086550729063585


In [2]:
(*question 4*)
let rec v a n =
    if n = 0 then a
    else let c = v a (n-1) in 1.0/.(2.0)*.(c +. a/.c);;

v 4. 100;;

val v : float -> int -> float = <fun>


- : float = 2.


In [3]:
v 100. 1000;;
v 25. 1000;;
v 9.0 1000;;

- : float = 10.


- : float = 5.


- : float = 3.


##### 
(*question 5*)
On conjecture que cette suite nous permet de connaître la racine d'un nombre.*

Démonstration: 

La suite est définie par récurrence, on peut donc l'écrire sous la forme u(n+1) = f(u(n)).
Pour trouver la limite de u(n), on résout l'équation l = f(l)
l = 1/2(l + a/l)
donc l/2 = a/(2l)
donc l^2 = a
donc l = sqrt(a)

on a donc bien démontré que la limite de cette suite est racine de a.

In [17]:
(*question 6*)
let rec euclide a b =
    if b = 0 then a
    else euclide b (a mod b);;

euclide 12 6;;
euclide 13 7;;

val euclide : int -> int -> int = <fun>


- : int = 6


- : int = 1


## Exponentiation rapide

1. Écrire une fonction récursive `puissance` naïve (c'est à dire très simple) telle que `puissance a n` renvoie $a^n$. Combien effectue t-elle de multiplications (en fonction de n) ?
2. Écrire une fonction récursive `exp_rapide` pour calculer $a^n$, en utilisant les relations suivantes :
$$ 
\begin{cases} 
a^n = (a^{\frac{n}{2}})^2 ~~~~~~~~~\text{si }n\text{ est pair}\\
a^{n} = a \times (a^{\frac{n-1}{2}})^2 ~~~~~\text{sinon}
\end{cases}
$$
**Attention** : pour mettre au carré, utiliser une variable plutôt que faire 2 appels récursifs, ce qui augmenterait beaucoup le nombre d'opérations.  
**Remarque** : On montrera plus tard que cette 2ème version demande de l'ordre de $\ln(n)$ multiplications seulement.

In [24]:
(*question 1*)
let rec puissance a n =
    if n = 0 then 1
    else a*puissance a (n-1);;

puissance 4 4;;

val puissance : int -> int -> int = <fun>


- : int = 256


In [25]:
(*question 2*)
let rec exp_rapide a n =
    if n = 0 then 1
    else
        if n mod 2 = 0 then let c = puissance a (n/2) in c*c
        else let d = puissance a ((n-1)/2) in a*d*d;;

exp_rapide 4 4;;

val exp_rapide : int -> int -> int = <fun>


- : int = 256


# Accumulateur

On a vu dans le cours sur la récursivité (avec l'exemple de la suite de Fibonacci) qu'un accumulateur est un argument que l'on ajoute à une fonction pour calculer sa valeur de retour.  
1. Écrire une fonction `fact` telle que `fact acc n` renvoie `n`!, en utilisant `acc` comme accumulateur. Voici à quoi va ressemble `fact` :
```ocaml
let rec fact acc n =
    if n = 0 then acc  (* on renvoie l'accumulateur qui contient le résultat *)
    else fact ... (* appel récursif en modifiant l'accumulateur *)
```
2. En utilisant `fact` et l'application partielle de fonction, définir `f : int -> int` renvoyant la factoielle d'un entier.

**Remarque** : le but ici est juste de vous entraîner à savoir utiliser un accumulateur, qui sont parfois utiles (comme pour la fonction `fibo` du cours). En DS ou concours on évitera d'utiliser un accumulateur lorsqu'il y en a pas besoin (comme pour la fonction `fact`...), car cela rend le code plus compliqué.  

In [27]:
(*question1*)
let rec fact n =
    if n = 0 then 1
    else n * fact (n-1);;

fact 5;;

val fact : int -> int = <fun>


- : int = 120


In [34]:
(*question 2*)
let rec fact acc n =
    if n = 0 then acc
    else fact (n*acc) (n-1);;

fact 1 5;;

val fact : int -> int -> int = <fun>


- : int = 120


## Temps de vol de la suite de Syracuse

La suite de Syracuse d'un entier $a$ est définie par :  
$$u_0 = a$$
$$u_{n+1} =
\begin{cases} 
     \frac{u_n}{2}, \text{si } u_n \text{ est pair}\\
    3u_n + 1, \text{sinon}\\
\end{cases}$$

Le temps de vol de $(u_n)_n$ est le plus petit entier $t$ tel que $u_t = 1$.  

1. Écrire une fonction récursive `temps_vol` telle que `temps_vol a` renvoie le temps de vol de $(u_n)_n$ (où $u_0 = a$).

In [11]:
let rec temps_vols a n =
    if a = 1 then (a,n)
    else if a mod 2 = 0  then temps_vols (a/2) (n+1)
    else temps_vols (3*a+1) (n+1);;

temps_vols 18 0;;
temps_vols 100 0;;
temps_vols 97 0;;

val temps_vols : int -> int -> int * int = <fun>


- : int * int = (1, 20)


- : int * int = (1, 25)


- : int * int = (1, 118)


## Fonction mystérieuse

Tester la fonction suivante pour des $n$ entre $0$ et $100$, conjecturer un théorème et prouvez-le par récurrence.

In [1]:
let rec f n =
    if n > 100 then n - 10 
    else f (f (n + 11)) ;;

val f : int -> int = <fun>


## Fonctions mutuellement récursives

Il est possible de définir simultanément deux fonctions `f` et `g`, dépendant l'une de l'autre avec `and` :
```ocaml
let rec f x = ... and g y = ... in ...
```

Écrire deux fonctions `u` et `v` permettant de calculer le $n$ième terme des suites définies par :
$$u_0 = 2$$
$$v_0 = 3$$
$$u_{n + 1} = u_n - u_n v_n$$
$$v_{n + 1} = v_n + u_n v_n$$

In [1]:
let rec u n =
    if n = 0 then 2
    else let a = u(n-1) in a - a*v(n-1)
    and v n =
    if n = 0 then 3
    else let b = v(n-1) in b +b*u(n-1);;

u 2;;
v 2;;

val u : int -> int = <fun>
val v : int -> int = <fun>


- : int = 32


- : int = -27


## Retour sur les tours de Hanoi

**Remarque** : le problème des tours de Hanoi a déjà été vu en stage, mais on le refait ici en OCaml. En outre, il est utile de réactiver sa mémoire pour se souvenir des méthodes et il m'arrivera pendant l'année de redonner des exercices déjà posés.

![](../img/hanoi.png)

$n$ disques sont posés sur la tige à gauche. L'objectif est de déplacer tous les disques sur la tige à droite :

![](../img/hanoi2.png)

Règles du jeu :

- On ne peut déplacer qu'un disque à la fois (celui tout en haut), sur une autre tige.
- Il est interdit de poser un disque sur un autre plus petit.

Exemple de premier déplacement valide :

![](../img/hanoi3.png)

On souhaite écrire une fonction récursive `hanoi` telle que `hanoi n tige1 tige2` affiche une suite de déplacements (avec des `print_int`) permettant de déplacer $n$ disques depuis `tige1` vers `tige2`. On supposera que les tiges sont numérotées 0, 1, 2 (de gauche à droite).

1. Supposons que vous sachiez déplacer $n-1$ disques d'une tige à une autre. Comment déplacer $n$ disques d'une tige à une autre ?
2. Écrire `hanoi`.

In [2]:
    let rec hanoi n tige0 tige2 tige1 = 
        if n <= 1 then 
             Format.printf "%d ==> %d\n" tige0 tige2
        else
             (hanoi (n-1) tige0 tige1 tige2;
             Format.printf "%d ==> %d\n" tige0 tige2;
             hanoi (n-1) tige1 tige2 tige0);;

hanoi 5 0 2 1;;
print_newline();;

val hanoi : int -> int -> int -> int -> unit = <fun>


- : unit = ()


0 ==> 2
0 ==> 1
2 ==> 1
0 ==> 2
1 ==> 0
1 ==> 2
0 ==> 2
0 ==> 1
2 ==> 1
2 ==> 0
1 ==> 0
2 ==> 1
0 ==> 2
0 ==> 1
2 ==> 1
0 ==> 2
1 ==> 0
1 ==> 2
0 ==> 2
1 ==> 0
2 ==> 1
2 ==> 0
1 ==> 0
1 ==> 2
0 ==> 2
0 ==> 1
2 ==> 1
0 ==> 2
1 ==> 0
1 ==> 2
0 ==> 2



- : unit = ()
